# 1) Datenvorverarbeitung

In [1]:
# Suppress harmless warnings from MLflow for cleaner output, if desired
import warnings
warnings.filterwarnings("ignore", category=DeprecationWarning)
warnings.filterwarnings("ignore", category=UserWarning)

In [2]:
import os

import kagglehub
import pandas as pd

In [3]:
# Download latest version of dataset
# link: https://www.kaggle.com/datasets/nikhil7280/weather-type-classification
path = kagglehub.dataset_download("nikhil7280/weather-type-classification")

complete_path = path + "/" + os.listdir(path)[0]

print("Path to dataset:", complete_path)

100%|██████████| 186k/186k [00:00<00:00, 537kB/s]

Extracting files...
Path to dataset: /root/.cache/kagglehub/datasets/nikhil7280/weather-type-classification/versions/1/weather_classification_data.csv


In [4]:
# Read the raw data file (csv file)
df = pd.read_csv(complete_path)

In [5]:
df.head()

,Temperature,Humidity,Wind Speed,Precipitation (%),Cloud Cover,Atmospheric Pressure,UV Index,Season,Visibility (km),Location,Weather Type
0,14.0,73,9.5,82.0,partly cloudy,1010.82,2,Winter,3.5,inland,Rainy
1,39.0,96,8.5,71.0,partly cloudy,1011.43,7,Spring,10.0,inland,Cloudy
2,30.0,64,7.0,16.0,clear,1018.72,5,Spring,5.5,mountain,Sunny
3,38.0,83,1.5,82.0,clear,1026.25,7,Spring,1.0,coastal,Sunny
4,27.0,74,17.0,66.0,overcast,990.67,1,Winter,2.5,mountain,Rainy


In [6]:
df.describe()

,Temperature,Humidity,Wind Speed,Precipitation (%),Atmospheric Pressure,UV Index,Visibility (km)
count,13200.000000,13200.000000,13200.000000,13200.000000,13200.000000,13200.000000,13200.000000
mean,19.127576,68.710833,9.832197,53.644394,1005.827896,4.005758,5.462917
std,17.386327,20.194248,6.908704,31.946541,37.199589,3.856600,3.371499
min,-25.000000,20.000000,0.000000,0.000000,800.120000,0.000000,0.000000
25%,4.000000,57.000000,5.000000,19.000000,994.800000,1.000000,3.000000
50%,21.000000,70.000000,9.000000,58.000000,1007.650000,3.000000,5.000000
75%,31.000000,84.000000,13.500000,82.000000,1016.772500,7.000000,7.500000
max,109.000000,109.000000,48.500000,109.000000,1199.210000,14.000000,20.000000


In [7]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 13200 entries, 0 to 13199
Data columns (total 11 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   Temperature           13200 non-null  float64
 1   Humidity              13200 non-null  int64  
 2   Wind Speed            13200 non-null  float64
 3   Precipitation (%)     13200 non-null  float64
 4   Cloud Cover           13200 non-null  object 
 5   Atmospheric Pressure  13200 non-null  float64
 6   UV Index              13200 non-null  int64  
 7   Season                13200 non-null  object 
 8   Visibility (km)       13200 non-null  float64
 9   Location              13200 non-null  object 
 10  Weather Type          13200 non-null  object 
dtypes: float64(5), int64(2), object(4)
memory usage: 1.1+ MB


In [8]:
# Prüfen, ob die Daten der Zielvariable ausgewogen sind
for target_val in df["Weather Type"].unique():
    print(f"{target_val} has {len(df[df['Weather Type'] == target_val])} samples")

Rainy has 3300 samples
Cloudy has 3300 samples
Sunny has 3300 samples
Snowy has 3300 samples


In [9]:
from sklearn.preprocessing import LabelEncoder

def encode_df(df, categorical_cols = ["Cloud Cover", "Season", "Location", "Weather Type"]):# Im dict werden alle LabelEncoder gespeichert
    label_encoders = {}

    for col in categorical_cols:
        # Für jede Spalte wird ein neuer LabelEncoder erstellt
        le = LabelEncoder()
        
        if col in df.columns:
            df[col] = le.fit_transform(df[col])
            label_encoders[col] = le

    # Ausgabe welcher Wert welche nummerische Repräsentation hat
    for col, encoder in label_encoders.items():
        print(f"\n{col} mapping:")
        for i, class_name in enumerate(encoder.classes_):
            print(f"  {class_name} -> {i}")

    return df, label_encoders

In [10]:
data, label_encoders = encode_df(df)


Cloud Cover mapping:
  clear -> 0
  cloudy -> 1
  overcast -> 2
  partly cloudy -> 3

Season mapping:
  Autumn -> 0
  Spring -> 1
  Summer -> 2
  Winter -> 3

Location mapping:
  coastal -> 0
  inland -> 1
  mountain -> 2

Weather Type mapping:
  Cloudy -> 0
  Rainy -> 1
  Snowy -> 2
  Sunny -> 3


In [11]:
import pickle

with open('../../data/day_4/label_encoders.pkl', 'wb') as f:
    pickle.dump(label_encoders, f)

In [12]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 13200 entries, 0 to 13199
Data columns (total 11 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   Temperature           13200 non-null  float64
 1   Humidity              13200 non-null  int64  
 2   Wind Speed            13200 non-null  float64
 3   Precipitation (%)     13200 non-null  float64
 4   Cloud Cover           13200 non-null  int64  
 5   Atmospheric Pressure  13200 non-null  float64
 6   UV Index              13200 non-null  int64  
 7   Season                13200 non-null  int64  
 8   Visibility (km)       13200 non-null  float64
 9   Location              13200 non-null  int64  
 10  Weather Type          13200 non-null  int64  
dtypes: float64(5), int64(6)
memory usage: 1.1 MB


In [13]:
# Speichern der Features, ohne die Zielvariable
X = data.drop(columns=["Weather Type"])
X.head()

,Temperature,Humidity,Wind Speed,Precipitation (%),Cloud Cover,Atmospheric Pressure,UV Index,Season,Visibility (km),Location
0,14.0,73,9.5,82.0,3,1010.82,2,3,3.5,1
1,39.0,96,8.5,71.0,3,1011.43,7,1,10.0,1
2,30.0,64,7.0,16.0,0,1018.72,5,1,5.5,2
3,38.0,83,1.5,82.0,0,1026.25,7,1,1.0,0
4,27.0,74,17.0,66.0,2,990.67,1,3,2.5,2


In [14]:
# Speichern der Zielvariable
y = data["Weather Type"]
y.head()

0    1
1    0
2    3
3    3
4    1
Name: Weather Type, dtype: int64

In [15]:
from sklearn.model_selection import train_test_split

In [16]:
# train_test_split teilt sowohl die Features als auch die Zielvariable automatisch in Trainings- und Testdaten auf
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42)

In [17]:
for target_val in y_train.unique():
    print(f"{target_val} has {len(y_train[y_train == target_val])} samples")

1 has 2653 samples
3 has 2659 samples
0 has 2649 samples
2 has 2599 samples


In [18]:
# Durch das Parameter stratify wird sichergestellt, dass die Verteilung der Zielvariable in den Trainings- und Testdaten gleich ist
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42, stratify=y)

In [19]:
for target_val in y_train.unique():
    print(f"{target_val} has {len(y_train[y_train == target_val])} samples")

2 has 2640 samples
3 has 2640 samples
1 has 2640 samples
0 has 2640 samples


In [20]:
print("Größe von X_train:", X_train.shape)
print("Größe von X_test:", X_test.shape)

Größe von X_train: (10560, 10)
Größe von X_test: (2640, 10)


In [21]:
y_test = pd.DataFrame(y_test)
y_train = pd.DataFrame(y_train)

In [22]:
# Speichern der Trainings- und Testdaten für spätere Reproduzierbarkeit und Evaluierung
X_train.to_parquet("../../data/day_4/X_train.parquet")
X_test.to_parquet("../../data/day_4/X_test.parquet")
y_train.to_parquet("../../data/day_4/y_train.parquet")
y_test.to_parquet("../../data/day_4/y_test.parquet")

# 2) Model training mit MLflow

In [23]:
# Modell auswählen
from sklearn.ensemble import RandomForestClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import confusion_matrix, precision_recall_curve, average_precision_score, accuracy_score, precision_score, recall_score, f1_score
from time import time
from sklearn.preprocessing import label_binarize

In [24]:
import seaborn as sns
import matplotlib.pyplot as plt

In [25]:
import mlflow
from mlflow.models import infer_signature
from mlflow.tracking import MlflowClient

In [26]:
mlflow.set_tracking_uri("http://mlflow_server:5000")
mlflow.set_experiment("Weather Classification Models")

2025/08/15 14:27:17 INFO mlflow.tracking.fluent: Experiment with name 'Weather Classification Models' does not exist. Creating a new experiment.


<Experiment: artifact_location='mlflow-artifacts:/1', creation_time=1755268037521, experiment_id='1', last_update_time=1755268037521, lifecycle_stage='active', name='Weather Classification Models', tags={}>

## Definieren der Modelle

In [27]:
rfc = RandomForestClassifier(
    n_estimators=75,
    max_depth=10,
    min_samples_split=2,
    min_samples_leaf=2,
    max_features="sqrt",
    random_state=42,
)

knn = KNeighborsClassifier(
    n_neighbors=7,
    weights="distance",
    algorithm="auto",
    leaf_size=35,
    p=2,
)

svc = SVC(
    C=1.0,
    kernel="rbf",
    degree=3,
    gamma="scale",
    coef0=0.0,
    shrinking=True,
    probability=True,
    tol=0.001,
    cache_size=200,
    class_weight=None,
    verbose=False,
    max_iter=-1,
    decision_function_shape="ovr",
    break_ties=False,
)

rfc_grid=RandomForestClassifier(random_state=42)

param_grid_rfc = { 
    'n_estimators': [75, 150],
    'max_features': ['sqrt', 'log2'],
    'max_depth' : [5, 10, 15],
    'min_samples_leaf': [2, 3],
    'criterion' :['gini', 'entropy']
}

grid_rfc = GridSearchCV(estimator=rfc_grid, param_grid=param_grid_rfc, cv=5, n_jobs=-1, verbose=2)

knn_grid=KNeighborsClassifier()

param_grid_knn = { 
    'n_neighbors': [3, 5, 7],
    'weights': ['uniform', 'distance'],
    'algorithm': ['auto', 'ball_tree', 'kd_tree', 'brute'],
    'leaf_size': [20, 30, 40],
    'p': [1, 2]
}

grid_knn = GridSearchCV(estimator=knn_grid, param_grid=param_grid_knn, cv=5, n_jobs=-1, verbose=2)

svc_grid=SVC(probability=True, random_state=42)

param_grid_svc = {
    'C': [0.1, 2, 10],
    'kernel': ['rbf', 'sigmoid'],
    'degree': [2, 3],
    'coef0': [0.0, 0.5]
}

grid_svc = GridSearchCV(estimator=svc_grid, param_grid=param_grid_svc, cv=5, n_jobs=-1, verbose=2)

## Helper Funktionen fürs Evaluieren

In [28]:
def plot_precision_recall_curve(model_name, model, target_classes, y_test_true, X_test_data):
    # Ensure y_test_true is 1D for label_binarize, then use original if needed
    y_test_bin = label_binarize(y_test_true.values.ravel(), classes=range(len(target_classes)))
    y_score = model.predict_proba(X_test_data)

    plt.figure(figsize=(10, 7))
    colors = ['blue', 'red', 'green', 'orange', 'purple', 'brown'] # More colors for more classes
    for i, color in zip(range(y_test_bin.shape[1]), colors[:y_test_bin.shape[1]]):
        precision, recall, _ = precision_recall_curve(y_test_bin[:, i], y_score[:, i])
        ap = average_precision_score(y_test_bin[:, i], y_score[:, i])
        plt.plot(recall, precision, color=color, lw=2,
                 label=f"{target_classes[i]} (AP = {ap:.2f})")

    plt.xlabel("Recall")
    plt.ylabel("Precision")
    plt.title(f"Precision-Recall Curve (One-vs-Rest) for model {model_name}")
    plt.legend(loc="lower left")
    plt.grid(True)
    plt.tight_layout()
    # Save plot to a temporary file
    plot_path = f"precision_recall_curve_{model_name}.png"
    plt.savefig(plot_path)
    plt.close() # Close plot to free memory
    return plot_path

In [29]:
def create_confusion_matrix(model_name, y_true, y_pred):
    cm = confusion_matrix(y_true, y_pred)
    plt.figure(figsize=(10, 7))
    plt.title(f'Confusion Matrix for {model_name}')
    sns.heatmap(cm, annot=True, fmt='d', cmap='Blues')
    plt.xlabel('Predicted')
    plt.ylabel('True')
    # Save plot to a temporary file
    plot_path = f"confusion_matrix_{model_name}.png"
    plt.savefig(plot_path)
    plt.close() # Close plot to free memory
    return plot_path

## MLflow Runs

In [30]:
def train_evaluate_log_model(
    model_obj,
    model_name,
    X_train_data,
    y_train_data,
    X_test_data,
    y_test_data,
    feature_columns,
    target_classes_list,
    registered_model_name
):
    """
    Trains, evaluates, and logs a machine learning model using MLflow.

    Args:
        model_obj: The scikit-learn estimator (e.g., RandomForestClassifier, GridSearchCV).
        model_name (str): A descriptive name for the model (e.g., "Base KNN", "Tuned SVC").
        X_train_data (pd.DataFrame): Training features.
        y_train_data (np.array): Training target labels (1D).
        X_test_data (pd.DataFrame): Test features.
        y_test_data (np.array): Test target labels (1D).
        feature_columns (list): List of feature column names for importance logging.
        target_classes_list (list): List of target class names for PR curve plotting.
        registered_model_name (str): The name to register the model under in MLflow Model Registry.
    """
    with mlflow.start_run(run_name=model_name):
        print(f"\n--- Starting MLflow run for: {model_name} ---")

        # Determine if it's a GridSearchCV object or a base estimator
        is_grid_search = isinstance(model_obj, GridSearchCV)
        trained_model = None # This will hold the final fitted model

        # Training/Tuning
        print(f"  Training/Tuning {model_name}...")
        start_time = time()
        model_obj.fit(X_train_data, y_train_data)
        end_time = time()
        duration_time = end_time - start_time

        if is_grid_search:
            print(f"  Hyperparameter tuning time: {duration_time:.2f} seconds")
            mlflow.log_metric("tuning_time_seconds", duration_time)
            print("  Best parameters found: ", model_obj.best_params_)
            mlflow.log_params(model_obj.best_params_) # Log the best parameters
            trained_model = model_obj.best_estimator_
        else:
            print(f"  Training time: {duration_time:.2f} seconds")
            mlflow.log_metric("training_time_seconds", duration_time)
            mlflow.log_params(model_obj.get_params())
            trained_model = model_obj

        # Log feature importances if available (for tree-based models)
        if hasattr(trained_model, 'feature_importances_') and feature_columns is not None:
            feature_importances = {f"{feature_columns[i]}": float(list(trained_model.feature_importances_)[i]) * 100 for i in range(len(feature_columns))}
            print("  Feature importances in percent:")
            print(feature_importances)
            mlflow.log_dict(feature_importances, "feature_importances.json")

        # Log and Register the model
        mlflow.sklearn.log_model(
            trained_model,
            "model", # This is the artifact path within the run
            input_example=X_train_data.head(1),
            registered_model_name=registered_model_name # This registers it to the Model Registry
        )
        print(f"  Model logged and registered as '{registered_model_name}'.")

        # Evaluation
        print(f"  Evaluating {model_name}...")
        y_pred = trained_model.predict(X_test_data)

        accuracy = accuracy_score(y_test_data, y_pred)
        precision = precision_score(y_test_data, y_pred, average='weighted', zero_division=0)
        recall = recall_score(y_test_data, y_pred, average='weighted', zero_division=0)
        f1 = f1_score(y_test_data, y_pred, average='weighted', zero_division=0)

        mlflow.log_metric("test_accuracy", accuracy)
        mlflow.log_metric("test_precision", precision)
        mlflow.log_metric("test_recall", recall)
        mlflow.log_metric("test_f1_score", f1)
        print(f"  Test Accuracy: {accuracy:.4f}")
        print(f"  Test Precision: {precision:.4f}")
        print(f"  Test Recall: {recall:.4f}")
        print(f"  Test F1-Score: {f1:.4f}")

        # Create and log Confusion Matrix
        cm_plot_path = create_confusion_matrix(model_name, y_test_data, y_pred)
        mlflow.log_artifact(cm_plot_path)
        os.remove(cm_plot_path)
        print("  Confusion matrix logged.")

        # Create and log Precision-Recall Curve
        pr_curve_plot_path = plot_precision_recall_curve(model_name, trained_model, target_classes_list, y_test, X_test_data)
        if pr_curve_plot_path: # Only log if plot was generated
            mlflow.log_artifact(pr_curve_plot_path)
            os.remove(pr_curve_plot_path)
            print("  Precision-Recall curve logged.")

        print(f"--- Finished MLflow run for: {model_name} ---")

In [31]:
y_train_flat = y_train.values.ravel()
y_test_flat = y_test.values.ravel()

In [ ]:
registered_model_names = {
    "RandomForest": "WeatherClassifier_RandomForest",
    "KNN": "WeatherClassifier_KNN",
    "SVC": "WeatherClassifier_SVC"
}

# For base models
train_evaluate_log_model(rfc, "Base RandomForest", X_train, y_train_flat, X_test, y_test_flat, X.columns, list(['Cloudy', 'Rainy', 'Snowy', 'Sunny']),
                         registered_model_name=registered_model_names["RandomForest"])
train_evaluate_log_model(knn, "Base KNN", X_train, y_train_flat, X_test, y_test_flat, None, list(['Cloudy', 'Rainy', 'Snowy', 'Sunny']),
                         registered_model_name=registered_model_names["KNN"])
train_evaluate_log_model(svc, "Base SVC", X_train, y_train_flat, X_test, y_test_flat, None, list(['Cloudy', 'Rainy', 'Snowy', 'Sunny']),
                         registered_model_name=registered_model_names["SVC"])

# For tuned models (GridSearchCV results)
train_evaluate_log_model(grid_rfc, "Tuned RandomForest (GridSearch)", X_train, y_train_flat, X_test, y_test_flat, X.columns, list(['Cloudy', 'Rainy', 'Snowy', 'Sunny']),
                         registered_model_name=registered_model_names["RandomForest"])
train_evaluate_log_model(grid_knn, "Tuned KNN (GridSearch)", X_train, y_train_flat, X_test, y_test_flat, None, list(['Cloudy', 'Rainy', 'Snowy', 'Sunny']),
                         registered_model_name=registered_model_names["KNN"])
train_evaluate_log_model(grid_svc, "Tuned SVC (GridSearch)", X_train, y_train_flat, X_test, y_test_flat, None, list(['Cloudy', 'Rainy', 'Snowy', 'Sunny']),
                         registered_model_name=registered_model_names["SVC"])

print("\nAll MLflow logging complete. Check your UI for detailed experiment results!")


--- Starting MLflow run for: Base RandomForest ---
  Training/Tuning Base RandomForest...
  Training time: 0.42 seconds
  Feature importances in percent:
{'Temperature': 20.675947680138076, 'Humidity': 4.353038947973376, 'Wind Speed': 2.7131267318697945, 'Precipitation (%)': 15.120352581298976, 'Cloud Cover': 9.97621379561131, 'Atmospheric Pressure': 10.265036261496297, 'UV Index': 14.133846956823536, 'Season': 5.023913988276159, 'Visibility (km)': 16.640508663657204, 'Location': 1.0980143928552575}


Successfully registered model 'WeatherClassifier_RandomForest'.
2025/08/15 14:27:21 INFO mlflow.store.model_registry.abstract_store: Waiting up to 300 seconds for model version to finish creation. Model name: WeatherClassifier_RandomForest, version 1
Created version '1' of model 'WeatherClassifier_RandomForest'.


  Model logged and registered as 'WeatherClassifier_RandomForest'.
  Evaluating Base RandomForest...
  Test Accuracy: 0.9148
  Test Precision: 0.9166
  Test Recall: 0.9148
  Test F1-Score: 0.9151
  Confusion matrix logged.
  Precision-Recall curve logged.
--- Finished MLflow run for: Base RandomForest ---
🏃 View run Base RandomForest at: http://mlflow_server:5000/#/experiments/1/runs/f952a45853ca4beea41437c3483daeb9
🧪 View experiment at: http://mlflow_server:5000/#/experiments/1

--- Starting MLflow run for: Base KNN ---
  Training/Tuning Base KNN...
  Training time: 0.01 seconds


Successfully registered model 'WeatherClassifier_KNN'.
2025/08/15 14:27:24 INFO mlflow.store.model_registry.abstract_store: Waiting up to 300 seconds for model version to finish creation. Model name: WeatherClassifier_KNN, version 1
Created version '1' of model 'WeatherClassifier_KNN'.


  Model logged and registered as 'WeatherClassifier_KNN'.
  Evaluating Base KNN...
  Test Accuracy: 0.8943
  Test Precision: 0.8947
  Test Recall: 0.8943
  Test F1-Score: 0.8944
  Confusion matrix logged.
  Precision-Recall curve logged.
--- Finished MLflow run for: Base KNN ---
🏃 View run Base KNN at: http://mlflow_server:5000/#/experiments/1/runs/7769ab27b0f94659b71c2079d82ffe68
🧪 View experiment at: http://mlflow_server:5000/#/experiments/1

--- Starting MLflow run for: Base SVC ---
  Training/Tuning Base SVC...
  Training time: 11.11 seconds


...

  Model logged and registered as 'WeatherClassifier_SVC'.
  Evaluating Base SVC...
  Test Accuracy: 0.8208
  Test Precision: 0.8260
  Test Recall: 0.8208
  Test F1-Score: 0.8202
  Confusion matrix logged.
  Precision-Recall curve logged.
--- Finished MLflow run for: Base SVC ---
🏃 View run Base SVC at: http://mlflow_server:5000/#/experiments/1/runs/ef4938f529a74271b0f15e5e0efa1017
🧪 View experiment at: http://mlflow_server:5000/#/experiments/1

--- Starting MLflow run for: Tuned RandomForest (GridSearch) ---
  Training/Tuning Tuned RandomForest (GridSearch)...
Fitting 5 folds for each of 48 candidates, totalling 240 fits
  Hyperparameter tuning time: 25.18 seconds
  Best parameters found:  {'criterion': 'gini', 'max_depth': 10, 'max_features': 'sqrt', 'min_samples_leaf': 3, 'n_estimators': 75}
  Feature importances in percent:
{'Temperature': 20.611755674922776, 'Humidity': 4.401025222658636, 'Wind Speed': 2.587499210027659, 'Precipitation (%)': 14.71803455707028, 'Cloud Cover': 10.185

Registered model 'WeatherClassifier_RandomForest' already exists. Creating a new version of this model...
2025/08/15 14:28:09 INFO mlflow.store.model_registry.abstract_store: Waiting up to 300 seconds for model version to finish creation. Model name: WeatherClassifier_RandomForest, version 2
Created version '2' of model 'WeatherClassifier_RandomForest'.


  Model logged and registered as 'WeatherClassifier_RandomForest'.
  Evaluating Tuned RandomForest (GridSearch)...
  Test Accuracy: 0.9159
  Test Precision: 0.9179
  Test Recall: 0.9159
  Test F1-Score: 0.9163
  Confusion matrix logged.
  Precision-Recall curve logged.
--- Finished MLflow run for: Tuned RandomForest (GridSearch) ---
🏃 View run Tuned RandomForest (GridSearch) at: http://mlflow_server:5000/#/experiments/1/runs/6d3a7691399844588b63c1f418085c66
🧪 View experiment at: http://mlflow_server:5000/#/experiments/1

--- Starting MLflow run for: Tuned KNN (GridSearch) ---
  Training/Tuning Tuned KNN (GridSearch)...
Fitting 5 folds for each of 144 candidates, totalling 720 fits
  Hyperparameter tuning time: 12.84 seconds
  Best parameters found:  {'algorithm': 'ball_tree', 'leaf_size': 20, 'n_neighbors': 7, 'p': 1, 'weights': 'uniform'}


Registered model 'WeatherClassifier_KNN' already exists. Creating a new version of this model...
2025/08/15 14:28:25 INFO mlflow.store.model_registry.abstract_store: Waiting up to 300 seconds for model version to finish creation. Model name: WeatherClassifier_KNN, version 2
Created version '2' of model 'WeatherClassifier_KNN'.


  Model logged and registered as 'WeatherClassifier_KNN'.
  Evaluating Tuned KNN (GridSearch)...
  Test Accuracy: 0.9038
  Test Precision: 0.9051
  Test Recall: 0.9038
  Test F1-Score: 0.9041
  Confusion matrix logged.
  Precision-Recall curve logged.
--- Finished MLflow run for: Tuned KNN (GridSearch) ---
🏃 View run Tuned KNN (GridSearch) at: http://mlflow_server:5000/#/experiments/1/runs/c05aeb0d8f7045cfb0666d39b51354e7
🧪 View experiment at: http://mlflow_server:5000/#/experiments/1

--- Starting MLflow run for: Tuned SVC (GridSearch) ---
  Training/Tuning Tuned SVC (GridSearch)...
Fitting 5 folds for each of 24 candidates, totalling 120 fits


## Champion Modell bestimmen

In [ ]:
client = MlflowClient()

CHAMPION_METRIC = "test_f1_score"
GLOBAL_CHAMPION_ALIAS = "champion"

print(f"--- MLflow Champion Promotion Script ---")
print(f"Champion will be selected based on metric: '{CHAMPION_METRIC}'")

try:
    # 1. Get all registered models
    registered_models = client.search_registered_models()
    if not registered_models:
        print("No registered models found. Exiting.")
        exit()

    print("\n--- Collecting All Model Versions and Metrics ---")
    all_versions_across_all_models = []

    for rm in registered_models:
        registered_model_name = rm.name
        print(f"  Processing Registered Model: '{registered_model_name}'")

        # Get all versions for the current registered model
        # Order by version descending to prioritize newer versions if metrics are tied
        model_versions = client.search_model_versions(f"name='{registered_model_name}'")

        if not model_versions:
            print(f"    No versions found for '{registered_model_name}'.")
            continue

        for mv in model_versions:
            run_id = mv.run_id
            version = mv.version
            aliases = mv.aliases # Now correctly fetching aliases

            try:
                # Fetch the run associated with this model version to get its metrics
                run = client.get_run(run_id)
                metric_value = run.data.metrics.get(CHAMPION_METRIC)

                if metric_value is not None:
                    version_info = {
                        "registered_model_name": registered_model_name,
                        "version": version,
                        "run_id": run_id,
                        "metric_value": metric_value,
                        "aliases": aliases
                    }
                    all_versions_across_all_models.append(version_info)
                    print(f"    Version {version} of '{registered_model_name}' (Run ID: {run_id[:8]}...): {CHAMPION_METRIC}={metric_value:.4f} (Aliases: {aliases})")
                else:
                    print(f"    Warning: Version {version} of '{registered_model_name}' (Run ID: {run_id[:8]}...) does not have metric '{CHAMPION_METRIC}'. Skipping this version.")

            except Exception as e:
                print(f"    Error fetching run {run_id} for version {version} of '{registered_model_name}': {e}. Skipping this version.")

    if not all_versions_across_all_models:
        print("No valid model versions with the specified metric found across all registered models. Exiting.")
        exit()

    # 2. Determine the single best version across ALL models
    global_best_version_info = max(all_versions_across_all_models, key=lambda x: x['metric_value'])

    best_registered_model_name = global_best_version_info['registered_model_name']
    best_version_number = global_best_version_info['version']
    best_metric_value = global_best_version_info['metric_value']
    best_run_id = global_best_version_info['run_id']


    print(f"\n--- Promoting Global Champion ---")
    print(f"Identified GLOBAL best version: Version {best_version_number} of '{best_registered_model_name}'")
    print(f"  Run ID: {best_run_id}")
    print(f"  {CHAMPION_METRIC}: {best_metric_value:.4f}")

    # 3. Manage the global champion alias
    # First, find any existing version that currently holds the GLOBAL_CHAMPION_ALIAS
    current_global_champion_version_info = None
    for mv_info in all_versions_across_all_models:
        if GLOBAL_CHAMPION_ALIAS in mv_info['aliases']:
            current_global_champion_version_info = mv_info
            break # Found the current champion, break the loop

    if current_global_champion_version_info is not None:
        old_champion_name = current_global_champion_version_info['registered_model_name']
        old_champion_version = current_global_champion_version_info['version']

        if (old_champion_name == best_registered_model_name and
            old_champion_version == best_version_number):
            print(f"  Version {best_version_number} of '{best_registered_model_name}' is already the '{GLOBAL_CHAMPION_ALIAS}'. No change needed.")
        else:
            print(f"  Removing '{GLOBAL_CHAMPION_ALIAS}' alias from old champion: Version {old_champion_version} of '{old_champion_name}'...")
            client.delete_model_version_alias(
                name=old_champion_name,
                alias=GLOBAL_CHAMPION_ALIAS,
                version=old_champion_version
            )
            print(f"  Alias '{GLOBAL_CHAMPION_ALIAS}' removed from Version {old_champion_version} of '{old_champion_name}'.")
            # Now set the alias on the new champion
            print(f"  Setting '{GLOBAL_CHAMPION_ALIAS}' alias on new champion: Version {best_version_number} of '{best_registered_model_name}'...")
            client.set_registered_model_alias(
                name=best_registered_model_name,
                alias=GLOBAL_CHAMPION_ALIAS,
                version=best_version_number
            )
            print(f"  Successfully set '{GLOBAL_CHAMPION_ALIAS}' alias on '{best_registered_model_name}' Version {best_version_number}!")
    else:
        # No existing global champion, just set the alias on the new best
        print(f"  No existing '{GLOBAL_CHAMPION_ALIAS}' found. Setting alias on: Version {best_version_number} of '{best_registered_model_name}'...")
        client.set_registered_model_alias(
            name=best_registered_model_name,
            alias=GLOBAL_CHAMPION_ALIAS,
            version=best_version_number
        )
        print(f"  Successfully set '{GLOBAL_CHAMPION_ALIAS}' alias on '{best_registered_model_name}' Version {best_version_number}!")


except Exception as e:
    print(f"\nAn error occurred during champion promotion: {e}")
    print("Please ensure your MLflow Tracking Server is running and accessible at the specified URI,")
    print("and that it's configured with a database backend for the Model Registry.")

print("\n--- Global Champion Promotion Script Finished ---")